In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file, regimes=['0.1', '0.2', '0.5', '1.0']):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        if root.name.rsplit("-", 1)[1] in regimes:
            metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>Evaluation - Searching temperature hyperparameter</h1>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>GlaS Dataset</h2>

In [15]:
# Unsupervised learning models are evaluate over the whole dataset and a fraction of data for backpropagating over BN layers, 
# while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation-new/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-BA1': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_ba1')),
    'H-UNet-BASE-SWTA-T-BA1': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ba1')),
    'H-UNet-BASE-SWTA-BA4': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_ba4')),
    'H-UNet-BASE-SWTA-T-BA4': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ba4')),
}

In [16]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [17]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-BA1,0,1.0,0.2,62.56,46.11,43.24,6.51
1,H-UNet-BASE-SWTA-BA1,0,5.0,0.2,67.98,52.05,31.50,4.15
2,H-UNet-BASE-SWTA-BA1,0,10.0,0.2,68.29,52.60,39.40,5.43
3,H-UNet-BASE-SWTA-BA1,0,20.0,0.2,69.43,53.78,27.95,3.74
4,H-UNet-BASE-SWTA-BA1,0,50.0,0.2,72.66,57.64,28.95,3.89
...,...,...,...,...,...,...,...,...
115,H-UNet-BASE-SWTA-T-BA4,4,5.0,0.2,65.40,49.23,32.38,4.55
116,H-UNet-BASE-SWTA-T-BA4,4,10.0,0.2,68.29,52.78,33.07,4.88
117,H-UNet-BASE-SWTA-T-BA4,4,20.0,0.2,69.22,53.48,23.33,3.37
118,H-UNet-BASE-SWTA-T-BA4,4,50.0,0.2,68.90,53.28,31.12,4.38


Dice           Jaccard            \
                                          mean       std    mean       std   
Model                  Inv Temp Regime                                       
H-UNet-BASE-SWTA-BA1   1.0      0.2     63.736  1.841176  48.076  2.227685   
                       5.0      0.2     66.166  2.556713  50.284  2.703531   
                       10.0     0.2     68.040  1.949679  52.666  2.100019   
                       20.0     0.2     68.946  1.134782  53.398  1.227424   
                       50.0     0.2     70.198  1.973733  54.862  2.248782   
                       100.0    0.2     69.272  1.894629  53.888  1.960783   
H-UNet-BASE-SWTA-BA4   1.0      0.2     61.686  1.918627  45.368  1.979209   
                       5.0      0.2     63.084  4.519268  47.022  4.849940   
                       10.0     0.2     65.130  3.555643  49.414  3.834538   
                       20.0     0.2     65.868  2.031285  50.142  2.423813   
                       50.0     0.2     67.154  2.043693  51.366  2.337901   
                       100.0    0.2     67.710  1.548967  51.956  1.601571   
H-UNet-BASE-SWTA-T-BA1 1.0      0.2     64.578  0.931300  48.740  1.247137   
                       5.0      0.2     67.616  0.950779  51.866  1.145199   
                       10.0     0.2     68.036  1.288189  52.346  1.678371   
                       20.0     0.2     68.344  1.628060  52.752  1.869564   
                       50.0     0.2     69.918  2.295402  54.626  2.569344   
                       100.0    0.2     70.834  3.205968  55.816  3.569087   
H-UNet-BASE-SWTA-T-BA4 1.0      0.2     63.678  1.476625  47.748  1.797518   
                       5.0      0.2     66.172  1.565621  50.212  1.724346   
                       10.0     0.2     67.510  1.426131  51.996  1.294712   
                       20.0     0.2     68.462  2.173378  52.818  2.460766   
                       50.0     0.2     67.666  2.455958  51.946  2.659413   
                       100.0    0.2     69.226  1.623724  53.726  2.017605   

                                       Hausdorff Distance             \
                                                     mean        std   
Model                  Inv Temp Regime                                 
H-UNet-BASE-SWTA-BA1   1.0      0.2                52.380   7.904451   
                       5.0      0.2                36.852  10.231357   
                       10.0     0.2                40.154   6.856619   
                       20.0     0.2                33.068   8.813116   
                       50.0     0.2                28.680   4.000412   
                       100.0    0.2                33.708   7.276151   
H-UNet-BASE-SWTA-BA4   1.0      0.2                44.224   5.362008   
                       5.0      0.2                44.664   5.215125   
                       10.0     0.2                42.610   6.565771   
                       20.0     0.2                40.632   3.999521   
                       50.0     0.2                35.970   4.032741   
                       100.0    0.2                33.246   9.672649   
H-UNet-BASE-SWTA-T-BA1 1.0      0.2                48.194   3.507453   
                       5.0      0.2                33.848   8.546737   
                       10.0     0.2                33.680   5.352014   
                       20.0     0.2                33.246   6.037129   
                       50.0     0.2                31.686   6.589843   
                       100.0    0.2                32.304   7.759602   
H-UNet-BASE-SWTA-T-BA4 1.0      0.2                52.082   4.773748   
                       5.0      0.2                37.722   5.584906   
                       10.0     0.2                40.286  12.060130   
                       20.0     0.2                29.200   4.747252   
                       50.0     0.2                32.730   4.596558   
                       100.0    0.2                30.814   4.8

In [18]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation-new/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice_fromBa1')),
    'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice_fromBa1')),
    'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice_fromBa4')),
    'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice_fromBa4')),
}

In [19]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [20]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1,0,1.0,0.2,78.33,65.40,28.29,3.88
1,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1,0,5.0,0.2,78.68,65.89,23.97,3.81
2,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1,0,10.0,0.2,82.09,70.37,20.11,2.52
3,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1,0,20.0,0.2,83.28,72.13,16.33,2.34
4,H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1,0,50.0,0.2,82.01,70.29,22.87,3.00
...,...,...,...,...,...,...,...,...
114,H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4,4,5.0,0.2,80.05,67.65,22.09,3.00
115,H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4,4,10.0,0.2,80.91,69.02,22.92,3.50
116,H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4,4,20.0,0.2,82.01,70.49,24.51,3.25
117,H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4,4,50.0,0.2,80.04,67.56,32.83,4.15


Dice  \
                                                                  mean   
Model                                         Inv Temp Regime            
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2     76.2600   
                                              5.0      0.2     78.5360   
                                              10.0     0.2     78.4140   
                                              20.0     0.2     79.9040   
                                              50.0     0.2     80.0660   
                                              100.0    0.2     81.0560   
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2     75.8140   
                                              5.0      0.2     79.0420   
                                              10.0     0.2     80.2920   
                                              20.0     0.2     78.6120   
                                              50.0     0.2     80.4620   
                                              100.0    0.2     79.6880   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2     76.6660   
                                              5.0      0.2     78.7740   
                                              10.0     0.2     78.9300   
                                              20.0     0.2     80.3280   
                                              50.0     0.2     80.2975   
                                              100.0    0.2     79.7620   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2     76.4420   
                                              5.0      0.2     78.8820   
                                              10.0     0.2     78.8000   
                                              20.0     0.2     80.0960   
                                              50.0     0.2     79.0540   
                                              100.0    0.2     78.7040   

                                                                         \
                                                                    std   
Model                                         Inv Temp Regime             
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2     2.551206   
                                              5.0      0.2     1.941811   
                                              10.0     0.2     2.982923   
                                              20.0     0.2     3.037438   
                                              50.0     0.2     2.903813   
                                              100.0    0.2     2.388520   
H-UNet-BASE-SWTA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2     1.765922   
                                              5.0      0.2     2.017318   
                                              10.0     0.2     1.911745   
                                              20.0     0.2     2.865078   
                                              50.0     0.2     1.228218   
                                              100.0    0.2     2.223065   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2     1.122310   
                                              5.0      0.2     2.110718   
                                              10.0     0.2     1.375482   
                                              20.0     0.2     1.349081   
                                              50.0     0.2     1.988372   
                                              100.0    0.2     1.940018   
H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2     1.268472   
                                              5.0      0.2     1.324111   
                                              10.0     0.2     1.687261   
                                              20.0     0.2     1.691975   
                                              50.0     0.2     1.639140   
                                              100.0    0.2     2.403077   

                                         

<h1>Evaluation - Hebbian Unsupervised Pretraining</h1>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [32]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation-new/runs"

INV_TEMP_GlaS=50          # to be set accordingly, used by SWTA
REGIMES=['0.2']

runs = {
    # UNET-based
    'H-UNet-HPCA-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ba1/inv_temp-1/regime-0.2').glob('run-*')),
    'H-UNet-HPCA-T-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ba1/inv_temp-1/regime-0.2').glob('run-*')),
    'H-UNet-HPCA-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ba4/inv_temp-1/regime-0.2').glob('run-*')),
    'H-UNet-HPCA-T-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ba4/inv_temp-1/regime-0.2').glob('run-*')),
    'H-UNet-BASE-SWTA-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ba1/inv_temp-{}/regime-0.2'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ba1/inv_temp-{}/regime-0.2'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ba4/inv_temp-{}/regime-0.2'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ba4/inv_temp-{}/regime-0.2'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [33]:
# Collect predictions scanning runs
#predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [34]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-BA1,0,50.0,0.2,72.66,57.64,28.95,3.89
1,H-UNet-BASE-SWTA-BA1,1,50.0,0.2,70.87,55.44,22.97,3.15
2,H-UNet-BASE-SWTA-BA1,2,50.0,0.2,67.24,51.39,32.36,4.70
3,H-UNet-BASE-SWTA-BA1,3,50.0,0.2,69.72,54.68,32.41,4.93
4,H-UNet-BASE-SWTA-BA1,4,50.0,0.2,70.50,55.16,26.71,3.68
5,H-UNet-BASE-SWTA-BA4,0,50.0,0.2,68.39,52.79,38.51,5.29
6,H-UNet-BASE-SWTA-BA4,1,50.0,0.2,66.96,51.00,31.65,4.35
7,H-UNet-BASE-SWTA-BA4,2,50.0,0.2,64.22,48.12,38.73,5.34
8,H-UNet-BASE-SWTA-BA4,3,50.0,0.2,66.56,50.62,39.47,5.52
9,H-UNet-BASE-SWTA-BA4,4,50.0,0.2,69.64,54.30,31.49,4.23


Dice           Jaccard            \
                                          mean       std    mean       std   
Model                  Inv Temp Regime                                       
H-UNet-BASE-SWTA-BA1   50.0     0.2     70.198  1.973733  54.862  2.248782   
H-UNet-BASE-SWTA-BA4   50.0     0.2     67.154  2.043693  51.366  2.337901   
H-UNet-BASE-SWTA-T-BA1 50.0     0.2     69.918  2.295402  54.626  2.569344   
H-UNet-BASE-SWTA-T-BA4 50.0     0.2     67.666  2.455958  51.946  2.659413   
H-UNet-HPCA-BA1        1.0      0.2     47.530  3.760943  31.871  3.434629   
H-UNet-HPCA-BA4        1.0      0.2     48.790  2.747162  32.920  2.664977   
H-UNet-HPCA-T-BA1      1.0      0.2     47.257  3.285592  31.533  3.004526   
H-UNet-HPCA-T-BA4      1.0      0.2     48.326  1.664926  32.390  1.535855   

                                       Hausdorff Distance             \
                                                     mean        std   
Model                  Inv Temp Regime                                 
H-UNet-BASE-SWTA-BA1   50.0     0.2                28.680   4.000412   
H-UNet-BASE-SWTA-BA4   50.0     0.2                35.970   4.032741   
H-UNet-BASE-SWTA-T-BA1 50.0     0.2                31.686   6.589843   
H-UNet-BASE-SWTA-T-BA4 50.0     0.2                32.730   4.596558   
H-UNet-HPCA-BA1        1.0      0.2                69.742  17.370862   
H-UNet-HPCA-BA4        1.0      0.2                61.252  19.444879   
H-UNet-HPCA-T-BA1      1.0      0.2                80.106  86.860660   
H-UNet-HPCA-T-BA4      1.0      0.2                51.830  15.945383   

                                       Average Surface Distance             
                                                           mean        std  
Model                  Inv Temp Regime                                      
H-UNet-BASE-SWTA-BA1   50.0     0.2                       4.070   0.736105  
H-UNet-BASE-SWTA-BA4   50.0     0.2                       4.946   0.606407  
H-UNet-BASE-SWTA-T-BA1 50.0     0.2                       4.474   0.970325  
H-UNet-BASE-SWTA-T-BA4 50.0     0.2                       4.712   0.881346  
H-UNet-HPCA-BA1        1.0      0.2                      12.326   2.574448  
H-UNet-HPCA-BA4        1.0      0.2                      10.098   2.838727  
H-UNet-HPCA-T-BA1      1.0      0.2                      18.071  27.624402  
H-UNet-HPCA-T-BA4      1.0      0.2                       8.794   2.214742

<h1>Evaluation - Data regime variations</h1>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

<h2>GlaS Dataset</h2>

In [37]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation-new/runs"

REGIMES = ['0.2']       # regimes to be considered
INV_TEMP_GlaS=50          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/unet_base/inv_temp-1').glob('regime-*')),
    #'UNet-256': list(Path(EXP_ROOT + '/experiment=glas/unet_base-256/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice_fromBa1/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice_fromBa1/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice_fromBa4/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice_fromBa4/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice_fromBa1/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice_fromBa1/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice_fromBa4/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice_fromBa4/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [38]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv', regimes=REGIMES) for k, v in runs.items() for r in v], ignore_index=True)

In [39]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,0,1.0,0.2,80.20,67.72,17.15,2.78
1,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,1,1.0,0.2,77.48,64.42,25.85,3.46
2,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,2,1.0,0.2,78.93,66.21,23.10,3.34
3,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,3,1.0,0.2,80.14,67.88,25.32,3.69
4,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,4,1.0,0.2,78.87,65.98,19.77,2.60
5,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,5,1.0,0.2,81.61,69.85,28.72,3.94
6,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,6,1.0,0.2,79.89,67.49,18.83,2.63
7,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,7,1.0,0.2,80.47,68.80,33.11,5.03
8,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,8,1.0,0.2,79.58,67.18,23.25,3.68
9,H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1,9,1.0,0.2,78.49,65.71,27.03,3.92


Dice            \
                                                               mean       std   
Model                                    Inv Temp Regime                        
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2     79.566000  1.165706   
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2     77.818000  2.426730   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2     78.482222  2.071043   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2     78.766000  2.863604   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1   50.0     0.2     80.066000  2.903813   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA4   50.0     0.2     80.462000  1.228218   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 50.0     0.2     80.297500  1.988372   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 50.0     0.2     79.054000  1.639140   

                                                            Jaccard            \
                                                               mean       std   
Model                                    Inv Temp Regime                        
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2     67.124000  1.590215   
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2     64.672000  3.296660   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2     65.662222  2.802752   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2     66.238000  3.656586   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1   50.0     0.2     67.958000  3.781147   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA4   50.0     0.2     68.358000  1.549119   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 50.0     0.2     68.315000  2.612566   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 50.0     0.2     66.390000  2.033495   

                                                         Hausdorff Distance  \
                                                                       mean   
Model                                    Inv Temp Regime                      
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2             24.213000   
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2             23.906000   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2             25.763333   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2             24.546000   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1   50.0     0.2             25.730000   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA4   50.0     0.2             24.098000   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 50.0     0.2             25.007500   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 50.0     0.2             28.028000   

                                                                    \
                                                               std   
Model                                    Inv Temp Regime             
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2     4.854315   
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2     2.919971   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2     4.384849   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2     3.113580   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1   50.0     0.2     4.149964   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA4   50.0     0.2     0.857887   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA1 50.0     0.2     3.172301   
H-UNet-SWTA-T-FT-FROM-BEST-DICE-FROM-BA4 50.0     0.2     4.115625   

                                                         Average Surface Distance  \
                                                                             mean   
Model                                    Inv Temp Regime                            
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA1   1.0      0.2                    3.507000   
H-UNet-HPCA-FT-FROM-BEST-DICE-FROM-BA4   1.0      0.2                    3.362000   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA1 1.0      0.2                    3.595556   
H-UNet-HPCA-T-FT-FROM-BEST-DICE-FROM-BA4 1.0      0.2                    3.596000   
H-UNet-SWTA-FT-FROM-BEST-DICE-FROM-BA1   50